In [3]:
import pandas as pd
import json

In [13]:
!pip install vllm

     ---------------------------------------- 0.0/874.1 kB ? eta -:--:--
     -- ---------------------------------- 61.4/874.1 kB 825.8 kB/s eta 0:00:01
     ------- ------------------------------ 174.1/874.1 kB 1.8 MB/s eta 0:00:01
     ------------- ------------------------ 307.2/874.1 kB 2.1 MB/s eta 0:00:01
     ------------------- ------------------ 450.6/874.1 kB 2.4 MB/s eta 0:00:01
     ------------------------------------ - 829.4/874.1 kB 3.5 MB/s eta 0:00:01
     -------------------------------------- 874.1/874.1 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      Traceback (most recent call last):
        File "C:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "C:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\andre\AppData\Local\Temp\pip-build-env-o7pm84rg\

In [4]:
# with open('datasets/hellaswag_ro/ro_validation.json', 'r', encoding='utf-8') as file:
#     data = json.load(file)

# df = pd.DataFrame(data)
df = pd.read_csv('datasets/hellaswag_ro/hellaswag_ro_smaller.csv', encoding='utf-8')


In [5]:
df

,ctx_a,ctx_b,ctx,endings,id,ind,activity_label,source_id,split,split_type,label
0,[header] Cum să cumpărați materialele didactic...,NaN,[header] Cum să cumpărați materialele didactic...,"['Întotdeauna, aveți un înregistrare scrisă a ...",hellaswag/validation/9199,44033,Education and Communications,wikihow~72483,val,indomain,2
1,Oamenii stau pe dealul de zăpadă.,oameni,Oamenii stau pe dealul de zăpadă. oameni,"['lucrează într-o clădire.', 'stau în găleți p...",hellaswag/validation/146,2059,Skiing,activitynet~v_Nkcc9kLDQz8,val,indomain,3
2,[header] Cum să ai grijă de părul blond decolo...,NaN,[header] Cum să ai grijă de părul blond decolo...,['Este mai bine să începeți de la rădăcină și ...,hellaswag/validation/7293,30038,Personal Care and Style,wikihow~49498,val,zeroshot,3
3,[header] Cum să îi ajutăm pe copii să nu se ma...,NaN,[header] Cum să îi ajutăm pe copii să nu se ma...,['[substeps] Nu vrei doar să spui „nu există n...,hellaswag/validation/8356,37690,Family Life,wikihow~62087,val,zeroshot,0
4,[header] Cum să faci pliuri la blugi [title] Î...,NaN,[header] Cum să faci pliuri la blugi [title] Î...,['[substeps] Îndoiește manșeta fiecărui picior...,hellaswag/validation/8745,40787,Personal Care and Style,wikihow~67085,val,zeroshot,3
...,...,...,...,...,...,...,...,...,...,...,...
2769,[header] Cum să scrii un plan strategic de IT ...,NaN,[header] Cum să scrii un plan strategic de IT ...,['[title] Elaborează un plan de strategie care...,hellaswag/validation/7425,31124,Computers and Electronics,wikihow~51285,val,indomain,2
2770,Două persoane stau într-o cameră. Încep să joa...,ei,Două persoane stau într-o cameră. Încep să joa...,['primesc o lovitură lungă din partea cealaltă...,hellaswag/validation/1020,14629,Playing squash,activitynet~v_s84BStnbfK8,val,indomain,1
2771,[header] Cum să eviți să fii jefuit [title] Ev...,NaN,[header] Cum să eviți să fii jefuit [title] Ev...,['[substeps] Cea mai bună modalitate de a evit...,hellaswag/validation/4875,12408,Personal Care and Style,wikihow~20488,val,zeroshot,1
2772,[header] Cum să faci apă de ovăz [title] Amest...,NaN,[header] Cum să faci apă de ovăz [title] Amest...,['[step] Lasă ovăzul în apa de înmuiere peste ...,hellaswag/validation/6557,24466,Food and Entertaining,wikihow~40340,val,indomain,3


In [33]:
fraction = 0.3
sampled_df = df.sample(frac=fraction, random_state=42).reset_index(drop=True)

sampled_df

,ctx_a,ctx_b,ctx,endings,id,ind,activity_label,source_id,split,split_type,label
0,[header] Cum să cumpărați materialele didactic...,,[header] Cum să cumpărați materialele didactic...,"[Întotdeauna, aveți un înregistrare scrisă a a...",hellaswag/validation/9199,44033,Education and Communications,wikihow~72483,val,indomain,2
1,Oamenii stau pe dealul de zăpadă.,oameni,Oamenii stau pe dealul de zăpadă. oameni,"[lucrează într-o clădire., stau în găleți pe o...",hellaswag/validation/146,2059,Skiing,activitynet~v_Nkcc9kLDQz8,val,indomain,3
2,[header] Cum să ai grijă de părul blond decolo...,,[header] Cum să ai grijă de părul blond decolo...,[Este mai bine să începeți de la rădăcină și s...,hellaswag/validation/7293,30038,Personal Care and Style,wikihow~49498,val,zeroshot,3
3,[header] Cum să îi ajutăm pe copii să nu se ma...,,[header] Cum să îi ajutăm pe copii să nu se ma...,[[substeps] Nu vrei doar să spui „nu există ni...,hellaswag/validation/8356,37690,Family Life,wikihow~62087,val,zeroshot,0
4,[header] Cum să faci pliuri la blugi [title] Î...,,[header] Cum să faci pliuri la blugi [title] Î...,[[substeps] Îndoiește manșeta fiecărui picior ...,hellaswag/validation/8745,40787,Personal Care and Style,wikihow~67085,val,zeroshot,3
...,...,...,...,...,...,...,...,...,...,...,...
2769,[header] Cum să scrii un plan strategic de IT ...,,[header] Cum să scrii un plan strategic de IT ...,[[title] Elaborează un plan de strategie care ...,hellaswag/validation/7425,31124,Computers and Electronics,wikihow~51285,val,indomain,2
2770,Două persoane stau într-o cameră. Încep să joa...,ei,Două persoane stau într-o cameră. Încep să joa...,[primesc o lovitură lungă din partea cealaltă....,hellaswag/validation/1020,14629,Playing squash,activitynet~v_s84BStnbfK8,val,indomain,1
2771,[header] Cum să eviți să fii jefuit [title] Ev...,,[header] Cum să eviți să fii jefuit [title] Ev...,[[substeps] Cea mai bună modalitate de a evita...,hellaswag/validation/4875,12408,Personal Care and Style,wikihow~20488,val,zeroshot,1
2772,[header] Cum să faci apă de ovăz [title] Amest...,,[header] Cum să faci apă de ovăz [title] Amest...,[[step] Lasă ovăzul în apa de înmuiere peste n...,hellaswag/validation/6557,24466,Food and Entertaining,wikihow~40340,val,indomain,3


In [34]:
sampled_df.to_csv("hellaswag_ro_smaller.csv", index=False, encoding='utf-8')

In [6]:
llama3_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Sunteți un asistent util, respectuos și onest. Veți primi 4 variante de propoziții având același context de început, dar continuări diferite. O singură variantă este corectă. Răspundeți doar cu varianta care completează contextul în mod corespunzător.<|eot_id|><|start_header_id|>user<|end_header_id|>

A. {option_a}
B. {option_b}
C. {option_c}
D. {option_d}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

llama2_prompt = """<s>[INST] <<SYS>>
Sunteți un asistent util, respectuos și onest. Veți primi 4 variante de propoziții având același context de început, dar continuări diferite. O singură variantă este corectă. Răspundeți doar cu varianta care completează contextul în mod corespunzător.
<</SYS>>

A. {option_a}
B. {option_b}
C. {option_c}
D. {option_d} [/INST]
"""

alpaca_prompt = """Mai jos veți primi 4 variante de propoziții având același context de început, dar continuări diferite. O singură variantă este corectă. Răspundeți doar cu varianta care completează contextul în mod corespunzător.

### Instrucțiune:
A. {option_a}
B. {option_b}
C. {option_c}
D. {option_d}

### Răspuns:
"""

In [7]:
import requests

def generate_response(prompt):
    url = "http://localhost:8000/generate"

    payload = {
        "prompt": prompt,
        "n": 1,
        "temperature": 0.95,
        "max_tokens": 1024
    }

    response = requests.post(url, json=payload)

    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code)
        return None

def get_chosen_answer(row):
    response_data = generate_response(row['formatted_prompt'])
    if response_data:
        answer = response_data['text'][0].split('\n')[-1]
        with open("hellaswag_responses_llama3_ro", 'a') as f:
            f.write(f"Question: {row['formatted_prompt']}\nAnswer: {answer}\n\n")
        return answer
    else:
        return None


In [13]:
# import ast
# df['endings'] = df['endings'].apply(ast.literal_eval)

def format_prompt(row, model='llama3'):
    # Combine the context with endings
    context = row['ctx']
    options = row['endings']
    
    # Use the correct prompt template
    if model == 'llama3':
        prompt_template = llama3_prompt
    elif model == 'llama2':
        prompt_template = llama2_prompt
    elif model == 'alpaca':
        prompt_template = alpaca_prompt
    else:
        raise ValueError("Unknown model specified")

    # Format the prompt
    prompt = prompt_template.format(
        context=context,
        option_a=context + options[0],
        option_b=context + options[1],
        option_c=context + options[2],
        option_d=context + options[3]
    )
    
    return prompt

In [15]:
df['formatted_prompt'] = df.apply(lambda row: format_prompt(row, model='llama3'), axis=1)

df['formatted_prompt'][4]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nSunteți un asistent util, respectuos și onest. Veți primi 4 variante de propoziții având același context de început, dar continuări diferite. O singură variantă este corectă. Răspundeți doar cu varianta care completează contextul în mod corespunzător.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nA. [header] Cum să faci pliuri la blugi [title] Îndoiește manșeta fiecărui picior până la o lungime convenabilă. [step] Rulează manșeta fiecărei piciorului de pantaloni pentru a ajusta lungimea manșetei. Aceasta îți va da și o idee despre cum va arăta manșeta atunci când partea de rulat va fi îndepărtată.[substeps] Îndoiește manșeta fiecărui picior de pantaloni la 45 de grade. Aici va fi locul în care îți vei face pliurile la blugi, potrivit ghidului de lungime pentru versiunea ta de blugi skinny.\nB. [header] Cum să faci pliuri la blugi [title] Îndoiește manșeta fiecărui picior până la o lungime convenabilă. [step] Rulează

In [ ]:
formatted_prompt = df['formatted_prompt'][1]

formatted_prompt

In [ ]:
df['response'] = df.apply(lambda row: get_chosen_answer(row), axis=1)


In [ ]:
df

In [ ]:
new_df = df['question',	'mc1_targets_choices', 'mc2_targets_choices',	'mc1_targets_labels',	'mc2_targets_labels', 'response'].copy()
# new_df.rename(columns={'answer': 'correct_answer'}, inplace=True)

In [ ]:
df.to_csv('hellaswag_llama3_ro.csv', index=False, encoding='utf-8')

In [11]:
from sentence_transformers import SentenceTransformer, util

# Load the multilingual sentence transformer
model_name = 'sentence-transformers/paraphrase-xlm-r-multilingual-v1'
model = SentenceTransformer(model_name)

# Example sentences
sentence1 = "Care țară mare cheltuiește cel mai mult pe turismul internațional? China cheltuiește cel mai mult pe turismul internațional."
sentence2 = "Care țară mare cheltuiește cel mai mult pe turismul internațional? China."

# Compute embeddings
embedding1 = model.encode(sentence1, convert_to_tensor=True)
embedding2 = model.encode(sentence2, convert_to_tensor=True)

# Compute cosine similarity
cosine_sim = util.cos_sim(embedding1, embedding2)
print(f"Cosine Similarity: {cosine_sim.item():.4f}")


Cosine Similarity: 0.9833


In [12]:
from sentence_transformers import SentenceTransformer, util

# Load the multilingual sentence transformer
model_name = 'BlackKakapo/stsb-xlm-r-multilingual-ro'
model = SentenceTransformer(model_name)

# Example sentences
sentence1 = "Care țară mare cheltuiește cel mai mult pe turismul internațional? China cheltuiește cel mai mult pe turismul internațional."
sentence2 = "Care țară mare cheltuiește cel mai mult pe turismul internațional? China."

# Compute embeddings
embedding1 = model.encode(sentence1, convert_to_tensor=True)
embedding2 = model.encode(sentence2, convert_to_tensor=True)

# Compute cosine similarity
cosine_sim = util.cos_sim(embedding1, embedding2)
print(f"Cosine Similarity: {cosine_sim.item():.4f}")


Cosine Similarity: 0.9831
